In [ ]:
import numpy as np
import pylab as plt
from matplotlib.animation import FuncAnimation
from matplotlib import rc
from IPython.display import HTML
rc('animation',html='html5')

All of the third-order interpolations, plus the set of weights corresponding to the five-point fifth-order interpolant.

In [ ]:
# leftmost (XXX*)
il = lambda s: 0.125 * (15*s[2] - 10*s[1] + 3*s[0])

# soft left (XX*X)
ic = lambda s: 0.125 * (3*s[3] +6*s[2] - s[1])

# soft right (X*XX)
ir = lambda s: 0.125 * (-s[4] + 6*s[3] + 3*s[2])

# collection of all three
intps = lambda s: np.array([il(s),ic(s),ir(s)])

# five-point centered
w0 = np.array([1/16,5/8,5/16])

Sharpness parameters and weights

In [ ]:
# from http://www.scholarpedia.org/article/WENO_methods
# more sophisticated sharpness parameter
betas= lambda s: np.array([
   4*s[0]**2-19*s[0]*s[1]+25*s[1]**2+11*s[0]*s[2]-31*s[1]*s[2]+10*s[2]**2,
   4*s[1]**2-13*s[1]*s[2]+13*s[2]**2+ 5*s[1]*s[3]-13*s[2]*s[3]+ 4*s[3]**2,
  10*s[2]**2-31*s[2]*s[3]+25*s[3]**2+11*s[2]*s[4]-19*s[3]*s[4]+ 4*s[4]**2
])/3

# get an [nx3] array of betas given an [nx5] stencil array
beta_arr = lambda stencils : np.array([betas(s) for s in stencils])

# construct a set of weights given a trio of betas
eps = 1e-6
def wts(bs):
    wgh = w0/(eps+bs)**2
    wgh /= np.sum(wgh)
    return wgh

Functions that take in an [nx5] array of stencil values and return length-n arrays of interpolant choice (ENO), weights, and interpolant values.

In [ ]:
# given an array of length-5 stencils, return the index [0,2] of the
# length-three stencil corresponding to the smallest beta
min_beta = lambda stencils: np.argmin(beta_arr(stencils),axis=1)

# given an array of length-5 stencils, return a [nx3] array of the
# weights as calculated from the betas
weights = lambda stencils: np.array([wts(b) for b in beta_arr(stencils)])

# given an array of length-5 stencils, return a [nx3] array of the
# third-order interpolations                                   
interps = lambda stencils: np.array([intps(s) for s in stencils])

ENO and WENO methods:

In [ ]:
######################################
# ENO method
#
# f: array of function values (padded with 2 ghost values on either side)
# v_fun: velocity function v = v(f)
# h: grid spacing step size
# fl: left boundary function values
######################################
def eno(f,v_fun,h,fl):
    
    # n x 5 array of stencils for each point
    stens = np.array([f[(i-2):(i+3)] for i in range(2,len(f)-2)])
    
    # set of interpolated values
    ints = interps(stens)
    
    # which index has least beta for each set of stencils
    ind = min_beta(stens)
    
    # interpolated values w/boundaries
    f_int = np.zeros(len(f)-3)
    f_int[0] = fl
    
    # plug in vals with smallest beta
    f_int[1:] = ints[np.arange(len(ints)),ind]
    
    # construct fluxes
    q_int = f_int * v_fun(f_int)
    
    # derivative
    return np.diff(q_int)/h

In [ ]:
######################################
# WENO method
#
# f: array of function values (padded with 2 ghost values on either side)
# v_fun: velocity function v = v(f)
# h: grid spacing step size
# fl: left boundary function values
######################################
def weno(f,v_fun,h,fl):
        
    # n x 5 array of stencils for each point
    stens = np.array([f[(i-2):(i+3)] for i in range(2,len(f)-2)])
    
    # set of interpolated values
    ints = interps(stens)
    
    # beta-dependent weights for each stencil
    wgh = weights(stens)
    
    # interpolated values w/boundaries
    f_int = np.zeros(len(f)-3)
    f_int[0] = fl
    
    # plug in dot product of values with weights
    f_int[1:] = np.sum(ints*wgh,axis=1)
    
    # construct fluxes
    q_int = f_int * v_fun(f_int)
    
    # derivative
    return np.diff(q_int)/h

Function for making an animated GIF of a solution

In [ ]:
####################################
# MAKE_ANIM
#
# v: velocity function v=v(f)
# f0: initial function f0=f0(x)
# N:  number of points
# L:  size of domain
# fl:  left boundary function value
# fr:  right boundary function value
# dt: time step size
# T: simulation length
# no: which non-oscillatory method to use
#####################################
def make_anim(v,f0,N=100,L=10,fl=0,fr=0,dt=0.0123,T=5,no=eno):
    
    # spatial values
    xx = (np.arange(N,dtype=np.float64)+0.5)*L/N
    
    # allocate / initialize function values
    fm = np.zeros(len(xx)+4,dtype=np.float64)
    fm[2:-2] = f0(xx)
    
    # allocate plot space
    fig,ax = plt.subplots()
    label = 'time = {:3.1f}'.format(0)
    ax.set_xlabel(label)
    plt.tight_layout()
    plt.close()
    orig, = ax.plot(xx,fm[2:-2],linewidth=4);
    line, = ax.plot(xx,fm[2:-2],linewidth=4);
    
    # how many frames to simulate?
    n_frs = int(T/dt)
    
    # update function for gif
    def update(i):
        
        # label current time
        label = 'time = {:3.1f}'.format(i*dt)
        ax.set_xlabel(label)

        # set function ghost values
        fm[:2]  = fl
        fm[-2:] = fr
        
        # update using d/dt = -dq/dx
        fm[2:-2] -= dt*no(fm,v,L/N,fl)
        
        # update line data in plot
        line.set_ydata(fm[2:-2]);
        return line,
    
    # create animation
    return FuncAnimation(fig,update,frames=np.arange(n_frs),interval=dt*1000,blit=True)

Velocity functions:

In [ ]:
# constant velocity
v_const_fun = lambda c : lambda f : c*np.ones(f.shape,np.float64)

# Burgers' equation
v_burg_fun = lambda s : lambda f : s*f

Initial conditions:

In [ ]:
# step / jump function
def f_jump(x,lo,hi,xc):
    
    f = lo*np.ones(x.shape)
    f[x>xc] += (hi-lo)
    
    return f

# customize
f_jump_fun = lambda l,h,c: lambda x: f_jump(x,l,h,c)

# triangular shape
def f_tri(x,lo,hi,height,base):
    
    avg = 0.5*(lo+hi)
    mid = np.logical_and(x>lo,x<hi)
    slope = 2*height/(hi-lo)
    f = np.zeros(x.shape)
    f[mid] = height - slope*np.abs(x[mid]-avg)
    return f + base

# customize
f_tri_fun = lambda l,h,ht,bs : lambda x: f_tri(x,l,h,ht,bs)

# sin wave hump
def f_sin(x,base,hgt,x_lo,x_hi):
    
    Dx = x_hi-x_lo
    mid = np.logical_and(x>x_lo,x<x_hi)
    f = np.ones(x.shape) * base
    f[mid] += hgt * np.sin(np.pi * (x[mid]-x_lo)/Dx)**2
    return f

# customize
f_sin_fun = lambda b,h,lo,hi : lambda x : f_sin(x,b,h,lo,hi)

Constant velocity, initial step function:

In [ ]:
# N 20 200

make_anim(v_const_fun(1),f_jump_fun(0,1,3),T=3,fr=1,N=100,no=weno)

Constant velocity, initially triangular:

In [ ]:
# N 40 200
# dt def 0.007
make_anim(v_const_fun(1),f_tri_fun(1,3,5,0),T=5,N=200,dt=0.007,L=10)

Burgers' velocity with triangle:

In [ ]:
# N 100
# dt 0.005
make_anim(v_burg_fun(0.2),f_tri_fun(1,3,5,2),T=6,dt=0.005,N=100,fl=2,fr=2)

Burgers' velocity and sine hump:

In [ ]:
# N 150
# dt 0.005
make_anim(v_burg_fun(0.2),f_sin_fun(1,10,0,2),fl=1,fr=1,N=150,dt=0.005)

In [ ]:
v_cars = lambda f:1-f

In [ ]:
make_anim(v_cars,jump_fun(0.8,0.95,8),fl=0.8,fr=0.95,T=2,L=10,N=40,dt=0.1,no=weno)